# ATM Cash Demand Forecasting
* Goal: Predict how much cash each ATM will need over different time horizons
* Objective: To build a time series model that accurately predicts the Cash_Demand_Next_Day for each ATM, enabling optimized cash replenishment, reduced operational costs, and minimized cash-out situations.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
df=pd.read_csv('/kaggle/input/atm-cash-demand-forecasting-and-management/atm_cash_management_dataset.csv')

## 1.Data Understanding

In [3]:
# Basic inspection
df.info()
print(df.describe())
print('missing data\n',df.isnull().sum()) # Check for missing values
df=df.drop_duplicates() #remove duplicates 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5658 entries, 0 to 5657
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ATM_ID                   5658 non-null   object
 1   Date                     5658 non-null   object
 2   Day_of_Week              5658 non-null   object
 3   Time_of_Day              5658 non-null   object
 4   Total_Withdrawals        5658 non-null   int64 
 5   Total_Deposits           5658 non-null   int64 
 6   Location_Type            5658 non-null   object
 7   Holiday_Flag             5658 non-null   int64 
 8   Special_Event_Flag       5658 non-null   int64 
 9   Previous_Day_Cash_Level  5658 non-null   int64 
 10  Weather_Condition        5658 non-null   object
 11  Nearby_Competitor_ATMs   5658 non-null   int64 
 12  Cash_Demand_Next_Day     5658 non-null   int64 
dtypes: int64(7), object(6)
memory usage: 574.8+ KB
       Total_Withdrawals  Total_Deposits  Holi

In [4]:
#Handle DateTime
df.head()

,ATM_ID,Date,Day_of_Week,Time_of_Day,Total_Withdrawals,Total_Deposits,Location_Type,Holiday_Flag,Special_Event_Flag,Previous_Day_Cash_Level,Weather_Condition,Nearby_Competitor_ATMs,Cash_Demand_Next_Day
0,ATM_0041,2022-04-25,Monday,Morning,57450,9308,Standalone,0,0,112953,Rainy,5,44165
1,ATM_0007,2023-11-24,Friday,Morning,72845,17896,Supermarket,0,0,115348,Clear,0,41249
2,ATM_0014,2022-08-27,Saturday,Morning,42957,12712,Supermarket,0,0,90731,Cloudy,1,29901
3,ATM_0029,2023-08-27,Sunday,Evening,43014,1375,Mall,0,0,88754,Rainy,5,44155
4,ATM_0028,2022-12-15,Thursday,Evening,36379,2938,Bank Branch,0,0,129312,Snowy,0,29784


* 5,658 records - good size for time series modeling
* No missing values - excellent data quality!
* Target variable: Cash_Demand_Next_Day 

In [5]:
#Ordinal encoding for Day_of_Week 
day_order = {"Monday": 1,"Tuesday": 2,"Wednesday": 3,"Thursday": 4,"Friday": 5,"Saturday": 6,"Sunday": 7}
df["Day_of_Week"] = df["Day_of_Week"].map(day_order)

In [6]:
df.head()

,ATM_ID,Date,Day_of_Week,Time_of_Day,Total_Withdrawals,Total_Deposits,Location_Type,Holiday_Flag,Special_Event_Flag,Previous_Day_Cash_Level,Weather_Condition,Nearby_Competitor_ATMs,Cash_Demand_Next_Day
0,ATM_0041,2022-04-25,1,Morning,57450,9308,Standalone,0,0,112953,Rainy,5,44165
1,ATM_0007,2023-11-24,5,Morning,72845,17896,Supermarket,0,0,115348,Clear,0,41249
2,ATM_0014,2022-08-27,6,Morning,42957,12712,Supermarket,0,0,90731,Cloudy,1,29901
3,ATM_0029,2023-08-27,7,Evening,43014,1375,Mall,0,0,88754,Rainy,5,44155
4,ATM_0028,2022-12-15,4,Evening,36379,2938,Bank Branch,0,0,129312,Snowy,0,29784


In [7]:
#print(df["Weather_Condition"].dropna().unique())
#print(df["Location_Type"].dropna().unique())
df["Time_of_Day"].dropna().unique()
df["ATM_ID"].dropna().unique()

array(['ATM_0041', 'ATM_0007', 'ATM_0014', 'ATM_0029', 'ATM_0028',
       'ATM_0006', 'ATM_0047', 'ATM_0040', 'ATM_0024', 'ATM_0045',
       'ATM_0003', 'ATM_0042', 'ATM_0048', 'ATM_0015', 'ATM_0030',
       'ATM_0020', 'ATM_0050', 'ATM_0031', 'ATM_0035', 'ATM_0034',
       'ATM_0038', 'ATM_0033', 'ATM_0037', 'ATM_0004', 'ATM_0016',
       'ATM_0018', 'ATM_0027', 'ATM_0025', 'ATM_0039', 'ATM_0023',
       'ATM_0046', 'ATM_0017', 'ATM_0001', 'ATM_0036', 'ATM_0002',
       'ATM_0011', 'ATM_0013', 'ATM_0021', 'ATM_0026', 'ATM_0019',
       'ATM_0010', 'ATM_0044', 'ATM_0012', 'ATM_0022', 'ATM_0009',
       'ATM_0032', 'ATM_0008', 'ATM_0049', 'ATM_0005', 'ATM_0043'],
      dtype=object)

In [8]:
df = pd.get_dummies(df, columns=["Weather_Condition", "Location_Type","Time_of_Day"])
#df = pd.get_dummies(df, columns=["Time_of_Day"])

In [9]:
df.head()

,ATM_ID,Date,Day_of_Week,Total_Withdrawals,Total_Deposits,Holiday_Flag,Special_Event_Flag,Previous_Day_Cash_Level,Nearby_Competitor_ATMs,Cash_Demand_Next_Day,...,Weather_Condition_Snowy,Location_Type_Bank Branch,Location_Type_Gas Station,Location_Type_Mall,Location_Type_Standalone,Location_Type_Supermarket,Time_of_Day_Afternoon,Time_of_Day_Evening,Time_of_Day_Morning,Time_of_Day_Night
0,ATM_0041,2022-04-25,1,57450,9308,0,0,112953,5,44165,...,False,False,False,False,True,False,False,False,True,False
1,ATM_0007,2023-11-24,5,72845,17896,0,0,115348,0,41249,...,False,False,False,False,False,True,False,False,True,False
2,ATM_0014,2022-08-27,6,42957,12712,0,0,90731,1,29901,...,False,False,False,False,False,True,False,False,True,False
3,ATM_0029,2023-08-27,7,43014,1375,0,0,88754,5,44155,...,False,False,False,True,False,False,False,True,False,False
4,ATM_0028,2022-12-15,4,36379,2938,0,0,129312,0,29784,...,True,True,False,False,False,False,False,True,False,False


In [10]:
#extract Date features
df["Date"] = pd.to_datetime(df["Date"])
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Day"] = df["Date"].dt.day
df["Week_of_Year"] = df["Date"].dt.isocalendar().week

# extract new features from the date 
df["Is_Month_End"] = df["Date"].dt.is_month_end.astype(int)
df["Is_Weekend"] = df["Date"].dt.weekday.isin([5, 6]).astype(int)

# remove the date column
df = df.drop(columns=["Date"])


In [11]:
df.head()

,ATM_ID,Day_of_Week,Total_Withdrawals,Total_Deposits,Holiday_Flag,Special_Event_Flag,Previous_Day_Cash_Level,Nearby_Competitor_ATMs,Cash_Demand_Next_Day,Weather_Condition_Clear,...,Time_of_Day_Afternoon,Time_of_Day_Evening,Time_of_Day_Morning,Time_of_Day_Night,Year,Month,Day,Week_of_Year,Is_Month_End,Is_Weekend
0,ATM_0041,1,57450,9308,0,0,112953,5,44165,False,...,False,False,True,False,2022,4,25,17,0,0
1,ATM_0007,5,72845,17896,0,0,115348,0,41249,True,...,False,False,True,False,2023,11,24,47,0,0
2,ATM_0014,6,42957,12712,0,0,90731,1,29901,False,...,False,False,True,False,2022,8,27,34,0,1
3,ATM_0029,7,43014,1375,0,0,88754,5,44155,False,...,False,True,False,False,2023,8,27,34,0,1
4,ATM_0028,4,36379,2938,0,0,129312,0,29784,False,...,False,True,False,False,2022,12,15,50,0,0


In [12]:
num_cols = ["Total_Withdrawals", "Total_Deposits", "Previous_Day_Cash_Level", "Nearby_Competitor_ATMs"]

scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

In [13]:
df.head()

,ATM_ID,Day_of_Week,Total_Withdrawals,Total_Deposits,Holiday_Flag,Special_Event_Flag,Previous_Day_Cash_Level,Nearby_Competitor_ATMs,Cash_Demand_Next_Day,Weather_Condition_Clear,...,Time_of_Day_Afternoon,Time_of_Day_Evening,Time_of_Day_Morning,Time_of_Day_Night,Year,Month,Day,Week_of_Year,Is_Month_End,Is_Weekend
0,ATM_0041,1,0.512802,-0.168308,0,0,0.629654,1.467003,44165,False,...,False,False,True,False,2022,4,25,17,0,0
1,ATM_0007,5,1.545870,1.592097,0,0,0.748686,-1.441098,41249,True,...,False,False,True,False,2023,11,24,47,0,0
2,ATM_0014,6,-0.459737,0.529458,0,0,-0.474787,-0.859478,29901,False,...,False,False,True,False,2022,8,27,34,0,1
3,ATM_0029,7,-0.455912,-1.794449,0,0,-0.573045,1.467003,44155,False,...,False,True,False,False,2023,8,27,34,0,1
4,ATM_0028,4,-0.901148,-1.474058,0,0,1.442702,-1.441098,29784,False,...,False,True,False,False,2022,12,15,50,0,0


In [14]:

df_encoded = df.copy()
target_means = df_encoded.groupby("ATM_ID")["Cash_Demand_Next_Day"].mean()
df_encoded["ATM_ID_TargetEnc"] = df_encoded["ATM_ID"].map(target_means)


df_encoded = df_encoded.drop(columns=["ATM_ID"])

In [15]:
df_encoded.head()

,Day_of_Week,Total_Withdrawals,Total_Deposits,Holiday_Flag,Special_Event_Flag,Previous_Day_Cash_Level,Nearby_Competitor_ATMs,Cash_Demand_Next_Day,Weather_Condition_Clear,Weather_Condition_Cloudy,...,Time_of_Day_Evening,Time_of_Day_Morning,Time_of_Day_Night,Year,Month,Day,Week_of_Year,Is_Month_End,Is_Weekend,ATM_ID_TargetEnc
0,1,0.512802,-0.168308,0,0,0.629654,1.467003,44165,False,False,...,False,True,False,2022,4,25,17,0,0,43193.000000
1,5,1.545870,1.592097,0,0,0.748686,-1.441098,41249,True,False,...,False,True,False,2023,11,24,47,0,0,41666.968000
2,6,-0.459737,0.529458,0,0,-0.474787,-0.859478,29901,False,True,...,False,True,False,2022,8,27,34,0,1,43190.242424
3,7,-0.455912,-1.794449,0,0,-0.573045,1.467003,44155,False,False,...,True,False,False,2023,8,27,34,0,1,43532.775510
4,4,-0.901148,-1.474058,0,0,1.442702,-1.441098,29784,False,False,...,True,False,False,2022,12,15,50,0,0,42078.532787


# Modeling

In [16]:
X = df_encoded.drop(columns=["Cash_Demand_Next_Day"])
y = df_encoded["Cash_Demand_Next_Day"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


num_cols = ["Total_Withdrawals", "Total_Deposits", "Previous_Day_Cash_Level", "Nearby_Competitor_ATMs", "ATM_ID_TargetEnc"]

scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])


model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)